# Fitting Single Sources

## Loading Example Data

The following data is a cutout of a bright galaxy in Abell-2744. The original data was aquired by the [Hubble Frontier Fields](https://frontierfields.org) team via the WFC3 instrument (`F105W` filter) and can be dirctly downloaded [here](https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0/hlsp_frontier_hst_wfc3-60mas_abell2744_f105w_v1.0_drz.fits). We also use a star from the field as a PSF model. 

We first use `astropy`'s ``CCDData`` to load the example data and visualize it through `matplotlib`.

In [ ]:
from astropy.nddata import CCDData

image = CCDData.read('data/abell_2744_cd_galaxy_f105w.fits.gz')

In [ ]:
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'

vmax = 0.05 # Use the image std as max and min of all plots 
vmin = - vmax 

plt.imshow(image.data, vmin=vmin, vmax=vmax)
plt.title("Galaxy in Abell 2744")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

Load PSF and normalize.

In [ ]:
from astropy.io import fits

PSF = fits.getdata('data/f105w_psf.fits.gz')
PSF = PSF / PSF.sum()

plt.imshow(PSF, vmin=0, vmax=PSF.std()/10)

## Compute Image Stats 

We will use the sigma clipped std as a threshold at the segmentation and deblending steps.


In [ ]:
from astropy.stats import sigma_clipped_stats

image_mean, image_median, image_stddev = sigma_clipped_stats(image.data, sigma=1)

## Segmentation, Deblending and Catalog 

Here we identity sources in the input image.


In [ ]:
from petrofit.segmentation import make_segments, deblend_segments, plot_segments
from photutils.segmentation import SourceCatalog 

# Define detect threshold
nsigma = np.ones_like(image.data)
nsigma[:, :] = image_stddev

# Define smoothing kernel
kernel_size = 3
fwhm = 3

# Min Source size (area)
npixels = 4**2

# Make segmentation map
segm = make_segments(image.data, nsigma=nsigma, 
                     kernel_size=kernel_size, 
                     fwhm=fwhm, npixels=npixels)

# Deblend segmentation map
segm_deblend = deblend_segments(image.data, segm, contrast=0.0, nlevels=50,
                                kernel_size=kernel_size, 
                                fwhm=fwhm, npixels=npixels)

# Make plots 
plot_segments(segm, image=image.data, vmax=vmax, vmin=vmin)
plt.show()

plot_segments(segm_deblend, image=image.data, vmax=vmax, vmin=vmin)
plt.show()

# Make catalog
cat = SourceCatalog(image.data, segm_deblend, wcs=image.wcs)

# Display source properties
print("Num of Targets:", len(cat))

# Convert to table
cat_table = cat.to_table()

cat_table[:10]

## Single Source Photometry 

The galaxy we are interested in fitting turns out to be the largest one in the image so we select that source to construct a curve of growth for it. We then define a radius list for the aperture photometry. Once `r_list` is defined, we run the `source_photometry` step.


In [ ]:
from petrofit.photometry import order_cat
from petrofit.photometry import make_radius_list

# Sort and get the largest object in the catalog
sorted_idx_list = order_cat(cat, key='area', reverse=True)
idx = sorted_idx_list[0] # index 0 is largest 
source = cat[idx]  # get source from the catalog 


r_list = make_radius_list(
    max_pix=250, # Max pixel to go up to
    n=100 # the number of radii to produce 
)

print(len(r_list))

In [ ]:
from petrofit.photometry import source_photometry

# Photomerty 
flux_arr, area_arr, error_arr = source_photometry(
    
    # Inputs 
    source, # Source (`photutils.segmentation.catalog.SourceCatalog`)
    image.data, # Image as 2D array 
    segm_deblend, # Deblended segmentation map of image
    r_list, # list of aperture radii  
    
    # Options 
    cutout_size=max(r_list)*2, # Cutout out size, set to double the max radius  
    bkg_sub=True, # Subtract background  
    sigma=1, sigma_type='clip', # Fit a 2D plane to pixels within 1 sigma of the mean
    plot=True, vmax=vmax, vmin=vmin, # Show plot with max and min defined above
)
plt.show()

## Estimating  Sersic Parameters 

### Center and Shape 

We estimate the center of the source (max pixel) and the elliptical parameters from the `SourceCatalog` object using helper functions.


In [ ]:
from petrofit.segmentation import get_source_ellip, get_source_theta, get_source_position

image_x_0, image_y_0 = get_source_position(source)

ellip = get_source_ellip(source)

theta = get_source_theta(source)

### Sersic Index

We use the Petrosian correction grid to estimate sersic index.

In [ ]:
from petrofit.petrosian import Petrosian, PetrosianCorrection

p = Petrosian(r_list, area_arr, flux_arr)

pc = PetrosianCorrection("data/concentration_index_grid_f105w_60mas.yaml")

In [ ]:
n = pc.estimate_n(
    p.r_half_light,
    p.concentration_index()[-1]
)

n

### Half Light Radius (r_eff)
We use the corrected Petrosian half light to estimate the `r_eff` 

In [ ]:
r_eff = p.r_half_light
r_eff

In [ ]:
corrected_epsilon = pc.estimate_epsilon(
    p.r_half_light,
    p.concentration_index()[-1]

)

corrected_p = Petrosian(r_list, area_arr, flux_arr, epsilon=corrected_epsilon)
              
r_eff = corrected_p.r_half_light
r_eff

r_total_flux is out of range so we use an epsilon that defines the last pixel as r_total_flux

In [ ]:
print(p.r_total_flux, corrected_p.r_total_flux)

In [ ]:
corrected_epsilon = r_list.max() / p.r_petrosian

corrected_p = Petrosian(r_list, area_arr, flux_arr, epsilon=corrected_epsilon)
              
r_eff = corrected_p.r_half_light
r_eff

### Amplitude at r_eff

We use photutils.isophote to fit find the flux at r_eff

In [ ]:
from photutils.isophote import EllipseGeometry, Ellipse

g = EllipseGeometry(image_x_0, image_y_0, 1., ellip, theta)
ellipse = Ellipse(image.data, geometry=g)

iso = ellipse.fit_isophote(r_eff)

amplitude = iso.intens
amplitude

## Zoom and Mask Image

In [ ]:
from astropy.nddata import Cutout2D
from petrofit.segmentation import masked_segm_image
cutout_size = 200

masked_image = masked_segm_image(source, image.data, segm_deblend, fill=None, mask_background=False)

fitting_image = Cutout2D(masked_image, (image_x_0, image_y_0), cutout_size)
fitting_image_unmasked = Cutout2D(image, (image_x_0, image_y_0), cutout_size)


x_0 = y_0 = cutout_size//2

print(np.unravel_index(fitting_image.data.argmax(), fitting_image.data.shape))

plt.imshow(fitting_image.data, vmin=0, vmax=image.data.std())
plt.title("Galaxy in Abell 2744")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

## Create Model

In [ ]:
from astropy.modeling import models 

center_slack = 4

sersic_model = models.Sersic2D(
    
        amplitude=amplitude,
        r_eff=r_eff,
        n=n,
        x_0=x_0,
        y_0=y_0,
        ellip=ellip, 
        theta=theta,
    
        bounds = {
            'amplitude': (0., None),
            'r_eff': (0, None),
            'n': (0, 10),
            'ellip': (0, 1),
            'theta': (-2*np.pi, 2*np.pi),
            'x_0': (x_0 - center_slack/2, x_0 + center_slack/2),
            'y_0': (y_0 - center_slack/2, y_0 + center_slack/2),
        },
)

In [ ]:
from petrofit.fitting.models import PSFModel

psf_sersic_model = PSFModel.wrap(sersic_model, psf=PSF, oversample=('x_0', 'y_0', 20, 4))

In [ ]:
psf_sersic_model.fixed['psf_pa'] = True

## Fit the PSF Model

In [ ]:
%%time

from petrofit.fitting.fitting import fit_model

fitted_model, _ = fit_model(
    fitting_image.data, psf_sersic_model,
    maxiter=10000,
    epsilon=1.4901161193847656e-08,
    acc=1e-09,
)

## Generate Model Image

In [ ]:
from petrofit.fitting.fitting import model_to_image

fitted_image_size = fitting_image.data.shape[0]
fitted_image_center = fitted_image_size // 2 

fitted_model_image = model_to_image(fitted_image_center, fitted_image_center, fitted_image_size, fitted_model)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=[24, 12])

ax[0].imshow(fitting_image_unmasked.data, vmin=vmin, vmax=vmax)
ax[0].set_title("Data")

ax[1].imshow(fitted_model_image, vmin=vmin, vmax=vmax)
ax[1].set_title("Model")

ax[2].imshow(fitting_image_unmasked.data - fitted_model_image, vmin=vmin, vmax=vmax)
ax[2].set_title("Residual")

plt.show()

## Compare Estimated and Fitted Parameters

In [ ]:
from petrofit.fitting.utils import print_model_params

print_model_params(fitted_model)

In [ ]:
assumptions = [
    amplitude,
    r_eff,
    n,
    x_0,
    y_0,
    ellip,
    theta,
    0 # psf_pa
]

print("fit\tassum\tparam_name")
for param_name, param_val, assumption in zip(fitted_model.param_names, fitted_model.parameters, assumptions):
    print("{:0.2f}\t{:0.2f}\t{}".format(param_val, assumption, param_name))